In [3]:
import autogen

config_list = {
    'model': 'llama3.1',
    'base_url': 'http://localhost:11434/v1',
    'api_key': 'ollama',
}

ModuleNotFoundError: No module named 'autogen'

In [ ]:
text_assistant = autogen.ConversableAgent(
    'assistant', 
    system_message='You are an agent designed to redact personal information from any text provided to you.' 
                   'Your task is to identify and replace any personal information with stars (**).'
                   'This includes, but is not limited to, names, phone numbers, email addresses, physical addresses, social security numbers, and any other personally identifiable information (PII).'
                   'Ensure that the redaction is consistent throughout the text.'
                   'An evaluation agent will review your work and provide you with feedback or tips on how to improve. Understand their feedback and make the necessary changes to your redacted text.'
                   'You must output the redacted text only and not address or respond to any agents directly.',
    llm_config=config_list, 
    human_input_mode='NEVER',
    code_execution_config=False
)

evaluation = autogen.ConversableAgent(
    'evaluation-agent',
    system_message='You are an evaluation agent tasked with reviewing text to ensure that all personal information has been correctly redacted.'
                   'Your job is to verify that any personal information (such as names, phone numbers, email addresses, physical addresses, social security numbers, and other personally identifiable information) has been replaced with stars (**).'
                   'If you find any personal information that has not been redacted or if the redaction is inconsistent, you should flag the specific issues and provide feedback.'
                   'Ensure that the redaction is thorough and meets the required standards.',
    llm_config=config_list,
    human_input_mode='NEVER',
    code_execution_config=False
)

user_proxy = autogen.UserProxyAgent(
    'user_proxy', 
    human_input_mode='NEVER',
    code_execution_config=False
)

chat_outline = {
    'message': f'To ensure that the requirement for transport of the academic year 2024-25 is met for all students seeking admission for the 1st year (CET Students) and students seeking continuation of the services of the Buses already being availed by the present 1st & 2nd year students, and also from the 3rd year students at SOE who will be migrating to Campus 4 to the 4th year from Campus 3, there is a requirement to have confirmation from the students'
                'To get the exact requirement from all the existing students, all the students of SOE, COPS, BPT, AHS & Nursing who require Transport Facility, to pay the Full Transport Fees by the end of their respective academic year 2023-24. This also applies to the present students who are availing the transport. The students who do not pay the transport fees will be considered as not requiring the transport arrangements for the academic year 2024-25. The Copy of the notification of the Charges with the respective routes applicable is attached to this email.'
                'Students of SOE 2nd, 3rd & 4th year, who require transport are required to apply online by logging into the ERP with their passwords and click on Transport and fill in the required data and pay the amount also online. All the students will submit their requirements for transport by 16 August 2024. '
                'Respective Dean / Principal\'s are requested to communicate the above to all the staff under them for strict compliance. For students, the same may be communicated to all through their respective groups or Google  Forms.',
    }


In [ ]:
conversation_state = 1
speakers = []
def text_redact_selection_func(speaker: autogen.AssistantAgent, groupchat: autogen.GroupChat):
    global conversation_state
    if speaker == user_proxy and conversation_state == 1:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant
    elif speaker == text_assistant and conversation_state == 2:
        conversation_state += 1
        speakers.append(evaluation.name)
        return evaluation
    elif speaker == evaluation and conversation_state == 3:
        conversation_state += 1
        speakers.append(text_assistant.name)
        return text_assistant

text_redaction_chat = autogen.GroupChat(
    agents=[user_proxy, text_assistant, evaluation],
    speaker_selection_method=text_redact_selection_func,
    messages=[],
    max_round=4
)

text_redaction_manager = autogen.GroupChatManager(
    groupchat=text_redaction_chat, llm_config=config_list
)

In [ ]:
text_redaction_chats = user_proxy.initiate_chat(
    text_redaction_manager,
    message=chat_outline['message']
)

user_proxy (to chat_manager):

To ensure that the requirement for transport of the academic year 2024-25 is met for all students seeking admission for the 1st year (CET Students) and students seeking continuation of the services of the Buses already being availed by the present 1st & 2nd year students, and also from the 3rd year students at SOE who will be migrating to Campus 4 to the 4th year from Campus 3, there is a requirement to have confirmation from the studentsTo get the exact requirement from all the existing students, all the students of SOE, COPS, BPT, AHS & Nursing who require Transport Facility, to pay the Full Transport Fees by the end of their respective academic year 2023-24. This also applies to the present students who are availing the transport. The students who do not pay the transport fees will be considered as not requiring the transport arrangements for the academic year 2024-25. The Copy of the notification of the Charges with the respective routes applicable is